## SRJ

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 3.0
symbols = ["H","H","H","H"]
electrons = 4
orbitals = 8
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 1*r_bohr], [0.0, 0.0, 2*r_bohr], [0.0,0.0,3*r_bohr],[0.0, 0.0, 4*r_bohr]])
H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-6g", method="pyscf")
hf_state = qchem.hf_state(electrons, qubits)




def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
            #print(f'The expectation value of {i} is', current_value)

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        old_grad.append(max_value)  #Appending the highest operator value to the grad list
        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation
        if j == 0:
            params = qml.numpy.zeros(len(ash_excitation), requires_grad=True)  #Parameters initialization
        else:
            params = np.append(params, 0.0) 


        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = e_th, options = {'disp': True, 'maxiter': 1e8, 'xtol':1e-8, 'ftol':1e-8})


        params= (result.x)
        energies.append(result.fun)
        print('Updated params are', params)
        print('Updated excitation are', ash_excitation)

        # New state generation - With the updated parameters
        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        # Append the new state to the states list
        states.append(ostate)

    return energies, params, ash_excitation, gs_state, H, qubits,old_grad


In [2]:
energies, params, ash_excitation, gs_state,H, qubits, old_grad = adaptvqe(adapt_it = 5, e_th=1e-12)
print('Energies:', energies)

Updated hf_state is [1 1 1 1 0 0 0 0]
HF state is -1.3325688880668922
The adapt iteration now is 0


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

Highest gradient excitation is [2, 3, 6, 7]
Current parameters: [1.12299215]
Current cost: -1.4300836708403124

Current parameters: [1.12299269]
Current cost: -1.4300836708403395

Optimization terminated successfully.
         Current function value: -1.430084
         Iterations: 2
         Function evaluations: 94
Updated params are [1.12299269]
Updated excitation are [[2, 3, 6, 7]]
0: ──X──────────────────────────────────┤  State
1: ──X──────────────────────────────────┤  State
2: ──X─╭FermionicDoubleExcitation(1.12)─┤  State
3: ──X─├FermionicDoubleExcitation(1.12)─┤  State
6: ────├FermionicDoubleExcitation(1.12)─┤  State
7: ────╰FermionicDoubleExcitation(1.12)─┤  State
The adapt iteration now is 1


/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

Highest gradient excitation is [0, 3, 5, 6]
Current parameters: [1.12299269 1.38120324]
Current cost: -1.5693729295009202

Current parameters: [1.09743884 1.37779489]
Current cost: -1.5694603734965438

Current parameters: [1.09716186 1.37775795]
Current cost: -1.5694603835520695

Optimization terminated successfully.
         Current function value: -1.569460
         Iterations: 3
         Function evaluations: 187
Updated params are [1.09716186 1.37775795]
Updated excitation are [[2, 3, 6, 7], [0, 3, 5, 6]]
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)─┤  State
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.38)─┤  State
2: ──X─╭FermionicDoubleExcitation(1.10)─├FermionicDoubleExcitation(1.38)─┤  State
3: ──X─├FermionicDoubleExcitation(1.10)─├FermionicDoubleExcitation(1.38)─┤  State
5: ────│────────────────────────────────├FermionicDoubleExcitation(1.38)─┤  State
6: ────├FermionicDoubleExcitation(1.10)─╰FermionicDoubleExcitation(1.38)─┤ 

...

Highest gradient excitation is [0, 1, 4, 5]
Current parameters: [1.09716184 1.37775795 1.44489268]
Current cost: -1.7420659504715434

Current parameters: [1.2016523  1.38132935 1.45784151]
Current cost: -1.7436329052003248

Current parameters: [1.20338933 1.3826631  1.45806194]
Current cost: -1.7436334410198824

Current parameters: [1.20339345 1.38269592 1.45818895]
Current cost: -1.7436334426139328

Optimization terminated successfully.
         Current function value: -1.743633
         Iterations: 4
         Function evaluations: 331
Updated params are [1.20339345 1.38269592 1.45818895]
Updated excitation are [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5]]
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.38)
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.38)
2: ──X─╭FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(1.38)
3: ──X─├FermionicDoubleExcitation(1.20)─├FermionicDoubleExcitation(1.38)
4: ────│────────────────────────────────│────

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

Highest gradient excitation is [1, 2, 4, 7]
Current parameters: [1.20339326 1.38269646 1.45819219 0.74667154]
Current cost: -1.795036626214571

Current parameters: [0.8736809  1.35355147 1.12296851 1.10548125]
Current cost: -1.83965191971211

Current parameters: [0.82174607 1.29738932 1.02657856 1.16076061]
Current cost: -1.8420601940271233

Current parameters: [0.82588743 1.28903995 1.01004783 1.16435748]
Current cost: -1.8421083826342342

Current parameters: [0.82832155 1.28818988 1.00813357 1.16401003]
Current cost: -1.8421101067227772

Current parameters: [0.82877639 1.28812854 1.00804508 1.16386032]
Current cost: -1.8421101508897062

Current parameters: [0.82882169 1.28813036 1.0080702  1.16384541]
Current cost: -1.8421101514225926

Optimization terminated successfully.
         Current function value: -1.842110
         Iterations: 7
         Function evaluations: 605
Updated params are [0.82882169 1.28813036 1.0080702  1.16384541]
Updated excitation are [[2, 3, 6, 7], [0, 3, 5, 

/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.virtualenvs/new_venv/lib/python3.11/site-packages/pennylane_lightning/core/_serialize.py:283: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)
/Users/pssrivathsansundar/.v

Highest gradient excitation is [1, 5]
Current parameters: [0.82881998 1.28813207 1.00807108 1.16383515 0.28091908]
Current cost: -1.85042660812017

Current parameters: [0.83224734 1.28907888 1.01501107 1.16482949 0.28091908]
Current cost: -1.8504351594894195

Current parameters: [0.83036895 1.28925141 1.01550129 1.16540845 0.28091908]
Current cost: -1.8504359188799806

Current parameters: [0.83013798 1.28924727 1.01541778 1.16552617 0.28091908]
Current cost: -1.8504359311255638

Optimization terminated successfully.
         Current function value: -1.850436
         Iterations: 4
         Function evaluations: 559
Updated params are [0.83013798 1.28924727 1.01541778 1.16552617 0.28091908]
Updated excitation are [[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5]]
0: ──X──────────────────────────────────╭FermionicDoubleExcitation(1.29)
1: ──X──────────────────────────────────├FermionicDoubleExcitation(1.29)
2: ──X─╭FermionicDoubleExcitation(0.83)─├FermionicDoubleExcitation(

In [4]:
params
ash_excitation

[[2, 3, 6, 7], [0, 3, 5, 6], [0, 1, 4, 5], [1, 2, 4, 7], [1, 5]]

In [6]:
import pennylane as qml

# Initialize device with 1000 shots
dev4 = qml.device("lightning.qubit", wires=qubits, shots=1000)

@qml.qnode(dev4)
def measurement(gs_state):
    qml.StatePrep(gs_state, wires=range(qubits))
    return qml.counts(all_outcomes = False)  # Samples all qubits

counts = measurement(gs_state)
print(counts)

{'00001111': 219, '00111100': 129, '01001011': 6, '01100110': 95, '01111000': 3, '10000111': 1, '10011001': 129, '10110100': 6, '11000011': 125, '11110000': 287}


## Generating |i + j > state

In [7]:
import pennylane as qml
import numpy as np

# Define coefficients and basis states
coeffs = np.sqrt(np.array([0.219, 0.287]))
bases = np.array([
    [0,0,0,0,1,1,1,1], 
    [1,1,1,1,0,0,0,0]
])

# Construct the 256-element state vector manually
state = np.zeros(2**8, dtype=complex)
for c, b in zip(coeffs, bases):
    idx = int("".join(map(str, b)), 2)  # Convert binary to integer index
    state[idx] = c


print('State now:',state)
# Normalize the state
state /= np.linalg.norm(state)

# Use only 8 qubits (no work wire)
wires = [0,1,2,3,4,5,6,7]
dev = qml.device("default.qubit", wires=wires)

@qml.qnode(dev)
def circuit():
    qml.StatePrep(state, wires=wires)  # Directly prepare the 256-state vector
    return qml.state()

# Verify
output_state = circuit()
print('OP state', output_state)
qubits = 8
dev4 = qml.device("lightning.qubit", wires=qubits, shots=1000)
@qml.qnode(dev4)
def measurement2():
    qml.StatePrep(output_state, wires=range(qubits))
    return qml.counts(all_outcomes = False)  # Samples all qubits

counts = measurement2()
print(counts)

print("State vector length:", len(output_state))  # 256 (2^8)
#print("Sample counts:", counts)  # Outputs 256 states


State now: [0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.46797436+0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.        +0